## 1. 라이브러리 호출

In [1]:
import pandas as pd
import numpy as np
import time
import re

from bs4 import BeautifulSoup
from datetime import datetime
from tqdm import tqdm
import requests

path = "/Users/kook/Desktop/"

## 2. 검색 키워드 및 개수 입력

In [2]:
# 필요한 키워드 입력
search_content = input("검색할 키워드를 입력해주세요: ")
max_news = int(input("\n몇 개의 뉴스를 크롤링할지 입력해주세요. ex) 1000(숫자만입력): "))

검색할 키워드를 입력해주세요:  제주도 에너지

몇 개의 뉴스를 크롤링할지 입력해주세요. ex) 1000(숫자만입력):  500


## 3. 검색 기간 설정

In [3]:
# 크롤링할 기간 설정
# ex) 2023년 금산인삼축제 기간: 2023.10.03 ~ 2023.10.13
# 따라서 축제 시작 전 3개월간의 뉴스를 크롤링
startday = ["2024.01.01"]
endday = ["2024.05.31"]

In [4]:
print("검색 항목 : ", search_content)
print("뉴스 최대 개수 : ", max_news)

검색 항목 :  제주도 에너지
뉴스 최대 개수 :  500


## 4. 크롤링, 파싱

In [5]:
# URL crawling 함수
# -> Redirect 되지않는 네이버뉴스 max_news개가 추출될때까지 크롤링을 계속하는 함수

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

def url_crawling(search_content, start_season, end_season, max_news):
    
    # 집합 형태로 저장해 중복 url 제거
    url_set = set()
    for start_day, end_day in zip(start_season, end_season):    
        for page in tqdm(range(1, 2000, 10)):
            response = requests.get(f"https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search_content}&start={page}&pd=3&ds={start_day}&de={end_day}", headers=headers)
            # page를 넘기다 page가 없으면 종료
            # 200은 HTTP 상태코드중 하나로 OK의 의미를 가짐. 요청이 성공적으로 처리되었음을 나타냄. 200이 아니라는것은 페이지가 없어 페이지를 넘길 수 없다는 의미
            if response.status_code != 200:
                print(f"페이지 {page//10}가 없습니다. Exiting.")
                break
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            ul = soup.select_one("div.group_news > ul.list_news")

            if ul is None:
                break
            li_list = ul.find_all('li')
            for li in li_list:
                a_list = li.select('div.news_area > div.news_info > div.info_group > a.info')
                for a_tag in a_list:
                    href = a_tag.get('href')
                    # href 속성값이 "n.news.naver.com"(네이버 뉴스)을 포함하는지 확인한다.
                    if "n.news.naver.com" in href:
                        try:
                            # request.head()로 추출한 url이 rediret되는지 확인한다. redirect 되지않은 url만 저장한다.
                            response = requests.head(href, allow_redirects=True)
                            if response.status_code == 200:
                                url_set.add(href)
                                # 원하는 개수의 기사가 모두 크롤링 되었으면 크롤링 종료
                                if len(url_set) >= max_news:
                                    return list(url_set)
                        except Exception as e:
                            print(f"An error occurred: {e}")
            time.sleep(1)

    return list(url_set)

In [6]:
url = url_crawling(search_content, startday, endday, max_news)
len(url)

100%|█████████████████████████████████████████| 200/200 [06:15<00:00,  1.88s/it]


397

## 5. 필요 내용 추출, 리스트 담기

In [7]:
news_url = url

In [8]:
# 신문사, 제목, 본문 추출
news_company = []
news_title = []
news_content = []

for url in tqdm(news_url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    company = soup.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_top > a > img[alt]")
    news_company.append(company['alt'] if company else 'None')
    title = soup.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    news_title.append(title.text if title else 'None')
    content = soup.select_one("article#dic_area")
    news_content.append(content.text if content else 'None')

100%|█████████████████████████████████████████| 397/397 [01:57<00:00,  3.37it/s]


In [9]:
# 데이터프레임 생성
columns = ["company", "url", "title", "content"]

data = {
    "company": news_company,
    "url": news_url,
    "title": news_title,
    "content": news_content
}

df_news = pd.DataFrame(data, columns=columns)
df_news.head()

,company,url,title,content
0,이데일리,https://n.news.naver.com/mnews/article/018/000...,"""뮤지컬 '어쩌면 해피엔딩' 브로드웨이 공연, 꿈만 같아요""",\n'윌휴' 콤비 박천휴·윌 애런슨의 K뮤지컬오는 10월 미국 뉴욕 벨라스코 극장 ...
1,연합뉴스,https://n.news.naver.com/mnews/article/001/001...,[전국 주요 신문 톱뉴스](29일 조간),"\n\t\t\t▲ 경향신문 = 윤 대통령, 이첩 당일 이종섭에 3차례 전화 ▲..."
2,세계일보,https://n.news.naver.com/mnews/article/022/000...,"제주·광주 “국립트라우마치유센터, 정부 지원 요구 공동 대응”","\n4·3-5·18 교두보, 민주평화·상생 맞손인권·산업·문화·인사교류 등 6개 과..."
3,중앙일보,https://n.news.naver.com/mnews/article/025/000...,"미국 지켜주는 미사일?…1기당 163억, 비싸고 말 많은 SM-3 [이철재의 밀담]",\n\t\t\t 말 많은 무기를 드디어 도입하기로 확정됐다. 해상 탄도미사일 요격...
4,강원도민일보,https://n.news.naver.com/mnews/article/654/000...,"한호연·이철규 후보, 현정부 국정평가 놓고 치열한 공방",\n22대 총선 동해·태백·삼척·정선 후보자 토론회22대 총선 강원 동해·태백·삼척...


## 6. 중복 뉴스 제거 및 Escape문자 제거

In [10]:
len(df_news)

397

In [11]:
# 내용 기준 중복 뉴스 제거
df_news = df_news.drop_duplicates(subset=['content'], keep='first')

In [12]:
len(df_news)

395

In [13]:
# Escape문자 제거
df_news['content'] = df_news['content'].apply(lambda x: re.sub(r'\s+', ' ', x))

In [14]:
df_news.head()

,company,url,title,content
0,이데일리,https://n.news.naver.com/mnews/article/018/000...,"""뮤지컬 '어쩌면 해피엔딩' 브로드웨이 공연, 꿈만 같아요""",'윌휴' 콤비 박천휴·윌 애런슨의 K뮤지컬오는 10월 미국 뉴욕 벨라스코 극장 개...
1,연합뉴스,https://n.news.naver.com/mnews/article/001/001...,[전국 주요 신문 톱뉴스](29일 조간),"▲ 경향신문 = 윤 대통령, 이첩 당일 이종섭에 3차례 전화 ▲ 국민일보 = 채상..."
2,세계일보,https://n.news.naver.com/mnews/article/022/000...,"제주·광주 “국립트라우마치유센터, 정부 지원 요구 공동 대응”","4·3-5·18 교두보, 민주평화·상생 맞손인권·산업·문화·인사교류 등 6개 과제..."
3,중앙일보,https://n.news.naver.com/mnews/article/025/000...,"미국 지켜주는 미사일?…1기당 163억, 비싸고 말 많은 SM-3 [이철재의 밀담]",말 많은 무기를 드디어 도입하기로 확정됐다. 해상 탄도미사일 요격 미사일인 SM-...
4,강원도민일보,https://n.news.naver.com/mnews/article/654/000...,"한호연·이철규 후보, 현정부 국정평가 놓고 치열한 공방",22대 총선 동해·태백·삼척·정선 후보자 토론회22대 총선 강원 동해·태백·삼척·...


In [15]:
df_news.to_excel("제주도 에너지 뉴스기사 크롤링.xlsx", index=False)